# Producer A


In [74]:
from kafka import KafkaProducer
import json
import time
import pandas as pd
from datetime import datetime

In [ ]:
# Load the data
df = pd.read_csv("data/camera_event_A.csv")

# Kafka settings
producer_id = 1
topic_name = "camera_events"
# kafka_host = "10.192.1.110:9092"
# kafka_host = "192.168.100.165:9092"
kafka_host = "10.192.0.245:9092"

batch_interval = 10

# Connect Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=[kafka_host],
    value_serializer=lambda v: json.dumps(v).encode("utf-8"),
    key_serializer=lambda k: str(k).encode("utf-8"),
    api_version=(0, 10)
)

# Group by batch_id
grouped = df.groupby("batch_id")

# Send batch-wise
for batch_id, group in grouped:
    send_time = datetime.utcnow().isoformat()
    timestamp_dt = datetime.fromisoformat(send_time)
    timestamp_ms = int(timestamp_dt.timestamp() * 1000)

    # Add extra metadata
    group = group.copy()
    group["producer_id"] = producer_id
    group["camera_id"] = 1
    group["send_time"] = send_time

    records = group.to_dict(orient="records")

    print(f"Sending batch_id={batch_id} | Records={len(records)} | Timestamp={send_time} | Sending from producer {producer_id}")

    # Send to Kafka (as batch payload)
    producer.send(
        topic=topic_name,
        key=producer_id,
        value=records,
        timestamp_ms=timestamp_ms
    )

    time.sleep(batch_interval)

print("All batches sent.")
